In [6]:
import pickle
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from typing import Dict, Tuple, List
from PIL import Image
from pathlib import Path

import tensorflow as tf

In [2]:
def read_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

def create_vgg16():
    # download VGG-16 with fully connected layers
    vgg = tf.keras.applications.vgg16.VGG16(
        include_top=True,
        weights="imagenet",
        input_shape=(224, 224, 3),
        pooling=None,
        classes=1000,
        classifier_activation="softmax",
    )
    # remove the classification layer
    new_model = tf.keras.models.Sequential()
    for layer in vgg.layers[:-1]:
        new_model.add(layer)
    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    x = tf.keras.applications.vgg16.preprocess_input(inputs)
    outputs = new_model(x)
    final_vgg = tf.keras.Model(inputs, outputs)
    return final_vgg

In [7]:
# Defining model logic

def most_common(arr) -> int:
  counts = np.bincount(arr.astype(int))
  return np.argmax(counts)


def map_to_int(x: List[str]) -> Tuple[np.array, Dict[int, str]]:
    """Map a list of strings to a list of ints and a dict mapping ints to strings."""
    mapping = {s: i for i, s in enumerate(set(x))}
    return np.array([mapping[s] for s in x]), mapping

def invert_dict(d: dict) -> dict:
  return {v: k for k, v in d.items()}

class DalleKNN:
  def __init__(self, labels: List[Tuple[str, str, str]]):
    self.labels = labels
    self.ages, self.age_map = map_to_int([x[0] for x in self.labels])
    self.genders, self.gender_map =  map_to_int([x[1] for x in self.labels])
    self.skin_tones, self.skin_tone_map =  map_to_int([x[2] for x in self.labels])
    assert len(np.unique(self.skin_tones)) == 10, "mismatch!"
    self.age_map = invert_dict(self.age_map)
    self.gender_map = invert_dict(self.gender_map)
    self.skin_tone_map = invert_dict(self.skin_tone_map)
    
  def fit(self, dalle_preds: np.ndarray):
    self.kdtree = KDTree(dalle_preds)
  
  def find_match(self, new_img, k=3) -> Tuple[str, str, str]:
    dist, idx = self.kdtree.query(new_img, k=k)
    age_pred = most_common(self.ages[idx])
    gender_pred = most_common(self.genders[idx])
    skin_tone_pred = most_common(self.skin_tones[idx])
    return self.age_map[age_pred], self.gender_map[gender_pred], self.skin_tone_map[skin_tone_pred]
  


In [8]:
# loading the model
MODEL = read_pickle('models/oii_unsupervised.pkl')

## Using the model
The model finds the nearest neighbour of a single image. The image have to be preprocessed by going through a VGG16 with the pretrained weights from imagenet. This can be initialized using the `create_vgg16()` function. This assumes images as np arrays of the shape (224, 224, 3). The below code illustrates loading the images.

In [ ]:
IMG_DIR = Path("test") # path to the images (change for your own images)
img_paths = IMG_DIR.glob("*.png")

# loading images
imgs = [Image.open(img_path).resize((224, 224)) for img_path in img_paths]
image_array = np.array([np.array(img) for img in imgs])

# loading the VGG-16 model
vgg = create_vgg16()
featurized_images = vgg.predict(image_array)


Now we can predict using our unsupervised model:

In [ ]:
preds = {"age": [], "gender": [], "skin_tone": []}

for test_img in featurized_images:
  agepred, genderpred, skinpred = MODEL.find_match(test_img)
  preds["age"].append(agepred)  
  preds["gender"].append(genderpred)  
  preds["skin_tone"].append(skinpred)  


## Evaluating the model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

def disparity_score(ytrue, ypred):
    cm = confusion_matrix(ytrue,ypred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    all_acc = list(cm.diagonal())
    return max(all_acc) - min(all_acc)

def evaluate_dict(test_df, pred_dict, score_func): 
  return {category: score_func(test_df[category], prediction) for category, prediction in pred_dict.items()}

test_labels = pd.read_csv("test/labels.csv") # path to the test labels

results = {}
results["accuracy"] = evaluate_dict(test_labels, pred_dict=preds, score_func=accuracy_score)
results["disparity"] = evaluate_dict(test_labels, pred_dict=preds, score_func=disparity_score)

In [ ]:
def getScore(results):
    acc = results['accuracy']
    disp = results['disparity']
    ad = 2*acc['gender']*(1-disp['gender']) + 4*acc['age']*(1-disp['age']**2) + 10*acc['skin_tone']*(1-disp['skin_tone']**5)
    return ad

title = "OII Gang Unsupervised"

submission = {
    'submission_name': title,
    'score': getScore(results),
    'metrics': results
}
submission